In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


In [3]:
import numpy as np
import pandas as pd

# 맨투맨만 뽑기

In [4]:
df = pd.read_csv('data/텍스트유사도.csv')
ctr = pd.read_csv('data/의류/total_ctr.csv')

df = df.set_index('content_id')
df = df[df.bg_rgb != '-1']
ctr= ctr.set_index('content_id')
ctr = ctr.loc[df.index]

man2man = df[df['content_cat_2'] == '맨투맨']
ctr_m2m = ctr.loc[man2man.index]

gender_ctr = ctr.groupby(['gender','content_id']).mean()

man2man['ctr_man'] = gender_ctr.loc[1]['ctr']
man2man['ctr_woman'] = gender_ctr.loc[2]['ctr']
man2man['ctr'] = ctr.groupby('content_id').mean()['ctr']

for col in ['ctr', 'ctr_man', 'ctr_woman', '제목-키워드']:
    man2man[col + '_ratio'] = man2man[col].apply(lambda x: 100*x/man2man[col].mean() -100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

In [5]:
man2man[['ctr', 'ctr_man', 'ctr_woman']].mean()

ctr          0.022293
ctr_man      0.023696
ctr_woman    0.020890
dtype: float64

In [6]:
def printer(col, threshold, df=df,ctr=ctr):
    print(f"{col} | {threshold}보다 큰거 : {round(ctr.loc[df[col]>threshold].mean()['ctr']*100,2)} %")
    print(f"{col} | {threshold}보다 작은거 : {round(ctr.loc[df[col]<=threshold].mean()['ctr']*100,2)} %")
    print('-'*50)

def classifier(col, threshold, df, ctr):
    low = round(ctr.loc[df[col]<=threshold].mean()['ctr']*100,2)
    high = round(ctr.loc[df[col]>threshold].mean()['ctr']*100,2)
    return low, high

def calculate_diff(low, high):
    if low > high:
        status = 'low'
        ratio = round((low - high)/high*100,2)
    else:
        status = 'high'
        ratio = round((high - low)/low*100,2)

    return status, ratio

In [7]:
def analysis_features(classify_features):
    result_df = pd.DataFrame(columns=['데이터', '컬럼', 'threshold', '구분', 'CTR'])
    diff_df = pd.DataFrame(columns=['데이터', '컬럼', 'threshold', '구분', 'ratio'])
    for i, (col, threshold) in enumerate(classify_features.items()):
        low, high = classifier(col, threshold, df, ctr)
        result_df.loc[4*i] = {'데이터': '전체', '컬럼': col, 'threshold': threshold, '구분': 'low', 'CTR': low}
        result_df.loc[4*i+1] = {'데이터': '전체', '컬럼': col, 'threshold': threshold, '구분': 'high', 'CTR': high}
        status, ratio = calculate_diff(low, high)
        diff_df.loc[2*i] = {'데이터': '전체', '컬럼': col, 'threshold': threshold, '구분': status, 'ratio': ratio}

        low, high = classifier(col, threshold, man2man, ctr_m2m)
        result_df.loc[4*i+2] = {'데이터': '맨투맨', '컬럼': col, 'threshold': threshold, '구분': 'low', 'CTR': low}
        result_df.loc[4*i+3] = {'데이터': '맨투맨', '컬럼': col, 'threshold': threshold, '구분': 'high', 'CTR': high}
        status, ratio = calculate_diff(low, high)
        diff_df.loc[2*i+1] = {'데이터': '맨투맨', '컬럼': col, 'threshold': threshold, '구분': status, 'ratio': ratio}

    return result_df, diff_df

In [8]:
threshold_dict = {
    'content_price': 30000,
    'content_views': 100,
    'content_likes': 5,
    'adv_item_count': 50,
    'adv_follower_count': 20,
    'adv_review_count': 20,
    'content_delivery_fee': 0,
    'content_used': 0,
    'tradable': 0,
    'background_ratio': 0.5,
    'bg_noise': 0.4,
    'nobg_noise': 0.4
}

In [9]:
result_df, diff_df = analysis_features(threshold_dict)

In [10]:
diff_df

,데이터,컬럼,threshold,구분,ratio
0,전체,content_price,30000,high,17.16
1,맨투맨,content_price,30000,low,11.06
2,전체,content_views,100,high,8.53
3,맨투맨,content_views,100,low,9.52
4,전체,content_likes,5,high,6.94
5,맨투맨,content_likes,5,low,3.23
6,전체,adv_item_count,50,high,3.72
7,맨투맨,adv_item_count,50,low,1.81
8,전체,adv_follower_count,20,high,8.41
9,맨투맨,adv_follower_count,20,low,8.61
